In [1]:
import numpy as np
import csv
import nltk
import itertools
from datetime import datetime
import rnn_with_numpy as rnn
reload(rnn)

<module 'rnn_with_numpy' from 'rnn_with_numpy.pyc'>

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/anderson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
SENTENCE_START = 'SETENCE_START'
SENTENCE_END = 'SENTENCE_END'
UNKNOWN_TOKEN = 'UNKNOWN_TOKEN'
VOCAB_SIZE = 10000

In [4]:
with open('/home/anderson/Dropbox/reddit_comments_filtered.csv', 'rt') as f:
    reader = csv.reader(f, skipinitialspace=True)
    print 'start reading comments! (wait for minutes...)'
    
    # Split full comments into sentences
    sentences = itertools.chain(
        *[nltk.sent_tokenize(x[0].decode('UTF-8').lower()) for x in reader])
    print 'Split Done'
    
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (SENTENCE_START, x, SENTENCE_END)
                    for x in sentences ]
    print 'START & END Done'

start reading comments! (wait for minutes...)
Split Done
START & END Done


In [5]:
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(st) for st in sentences ]

In [6]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print "Found %d unique words tokens." % len(word_freq.items())

Found 320177 unique words tokens.


In [7]:
vocab = word_freq.most_common(VOCAB_SIZE-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(UNKNOWN_TOKEN)
word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])

print "Using vocabulary size %d." % VOCAB_SIZE
print "The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1])

Using vocabulary size 10000.
The least frequent word in our vocabulary is 'craziest' and appeared 85 times.


In [8]:
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else UNKNOWN_TOKEN for w in sent]

In [9]:
print "'%s'" % sentences[28]
print "'%s'" % tokenized_sentences[28]

'SETENCE_START plus, there are things like epigenetics, which makes it even harder to figure out how the dna word look like or work. SENTENCE_END'
'[u'SETENCE_START', u'plus', u',', u'there', u'are', u'things', u'like', 'UNKNOWN_TOKEN', u',', u'which', u'makes', u'it', u'even', u'harder', u'to', u'figure', u'out', u'how', u'the', u'dna', u'word', u'look', u'like', u'or', u'work', u'.', u'SENTENCE_END']'


In [10]:
# Create the training data
x_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

print x_train[0]
print y_train[0]

[0, 496]
[496, 1]


In [11]:
reload(rnn)
model = rnn.RNNNumpy(VOCAB_SIZE)
o, s = model.forward_propagation(x_train[9])
p = model.predict(x_train[9])
print 'X train Size:', len(x_train[9])
print 'X train:', x_train[9]
print ' Output:', o.shape
print '  State:', s.shape
print 'Predicted:', p

(100, 10000)
(10000, 100)
(100, 10000)
X train Size: 13
X train: [0, 118, 14, 1603, 7, 92, 9999, 38, 6, 9999, 4787, 2680, 2]
 Output: (13, 10000)
  State: (14, 100)
Predicted: [4435 8091 3260  392 5663 1534 8414  185 2362 8414 3416 4146 8345]


In [12]:
model.cross_entropy(x_train[:1000], y_train[:1000])

(2, 10000)
(3, 100)
y[i] [496, 1]
[  9.99753079e-05   9.99480255e-05]
(12, 10000)
(13, 100)
y[i] [81, 531, 78, 36, 90, 22, 9999, 34, 12, 2784, 30, 1]
[  1.00042381e-04   1.00033745e-04   9.99713086e-05   1.00013558e-04
   9.99832983e-05   9.99727686e-05   9.99464360e-05   9.99929380e-05
   1.00017904e-04   9.99825572e-05   9.99838566e-05   9.99067517e-05]
(12, 10000)
(13, 100)
y[i] [3, 229, 537, 26, 19, 13, 11, 10, 9999, 1509, 2, 1]
[  9.99933118e-05   9.99958281e-05   1.00066136e-04   1.00021317e-04
   1.00012829e-04   9.99686610e-05   1.00036054e-04   1.00018824e-04
   9.99631574e-05   9.99694649e-05   1.00005678e-04   1.00034404e-04]
(18, 10000)
(19, 100)
y[i] [153, 85, 12, 547, 12, 211, 45, 1012, 6, 1273, 9999, 17, 11, 82, 9999, 452, 2, 1]
[  9.99809498e-05   1.00032334e-04   9.99559013e-05   9.99684905e-05
   1.00044233e-04   9.99291480e-05   9.99706120e-05   1.00044558e-04
   1.00026534e-04   1.00016374e-04   1.00016752e-04   1.00022027e-04
   1.00001837e-04   9.99778297e-05   1.

9.2103468900694594

In [13]:
model.bptt(x_train[:1000], y_train[26:27])

ValueError: could not broadcast input array from shape (100,2) into shape (100)